In [1]:
import numpy as np
import pandas as pd

import tensorflow as tf
from tensorflow.keras import layers

from sklearn.model_selection import train_test_split

In [2]:
path = [0, 1, 2, 3, 8, 10, 9, 7, 6, 5, 4, 0]
order = [0, 1, 2, 3, 10, 9, 8, 7, 4, 6, 5]
cumTime = [0, 14, 19, 22, 29, 39, 43, 47, 51, 56, 60, 73]
cumDistance = [0, 5.6, 7.4, 8.6, 10.7, 13.2, 14.2, 15.2, 16.1, 17.4, 18.1, 22.5]

In [3]:
def time_estimation(ggmap_time, numStop, numCustomer, timePerStop=5, timePerCustomer=2):

    estimation = ggmap_time * 1.1 + timePerStop * numStop + timePerCustomer * numCustomer
    estimation = estimation + (np.abs(np.random.normal(0, 0.2)) - 0.1) * estimation

    return np.round(estimation, 0)

In [4]:
df = pd.read_csv("./user-data.csv")
df = df.drop(['Unnamed: 0'], axis=1)

df.head()

,userID,orderID,addressID,parcelNumber
0,6500000001,1007,6,2
1,6500000002,1014,5,1
2,6500000003,1021,1,1
3,6500000004,1028,3,1
4,6500000005,1035,2,1


In [5]:
cumCustomer = [0] + list(df.groupby("addressID").count().userID.cumsum())

In [6]:
df["timeEstimation"] = df.apply(lambda x: time_estimation(cumTime[order[x.addressID]], order[x.addressID], cumCustomer[x.addressID]), axis=1)

In [7]:
df.head()

,userID,orderID,addressID,parcelNumber,timeEstimation
0,6500000001,1007,6,2,202.0
1,6500000002,1014,5,1,197.0
2,6500000003,1021,1,1,49.0
3,6500000004,1028,3,1,126.0
4,6500000005,1035,2,1,71.0


In [8]:
train, test = train_test_split(df, test_size=0.2)

In [9]:
train

,userID,orderID,addressID,parcelNumber,timeEstimation
34,6500000035,1245,1,3,45.0
79,6500000080,1560,5,5,209.0
40,6500000041,1287,8,1,238.0
30,6500000031,1217,5,1,193.0
60,6500000061,1427,1,2,58.0
...,...,...,...,...,...
44,6500000045,1315,4,1,182.0
41,6500000042,1294,4,2,214.0
10,6500000011,1077,7,1,264.0
67,6500000068,1476,1,1,46.0


In [42]:
df_new = pd.DataFrame({"index":np.array([0 for _ in range(100)])})
# df_new["timeEstimation"] = df.apply(lambda x: time_estimation(cumTime[order[x.addressID]], order[x.addressID], cumCustomer[x.addressID]), axis=1)
# df_new["ggmap_time"] = df.apply(lambda x: cumTime[order[x.addressID]], axis=1)
df_new.head()

,index
0,0
1,0
2,0
3,0
4,0


In [ ]:
df_new["timeEstimation"] = df.apply(lambda x: time_estimation(cumTime[order[x.addressID]], order[x.addressID], cumCustomer[x.addressID]), axis=1)


In [ ]:
df["timeEstimation"] = df.apply(lambda x: time_estimation(cumTime[order[x.addressID]], order[x.addressID], cumCustomer[x.addressID]), axis=1)
df["ggmap_time"] = 

In [26]:
train_features = train.copy()
train_labels = train_features.pop('timeEstimation')
train_features.pop("userID")
train_features = np.array(train_features)
address = train_features[:, 1]
train_features[:, 0] = np.array(list(map(lambda x: cumTime[order[x]], address)))
train_features[:, 1] = np.array(list(map(lambda x: order[x], address)))
train_features[:, 2] = np.array(list(map(lambda x: cumCustomer[x], address)))


test_features = test.copy()
test_labels = test_features.pop('timeEstimation')
test_features.pop("userID")
test_features = np.array(test_features)
address = test_features[:, 1]
test_features[:, 0] = np.array(list(map(lambda x: cumTime[order[x]], address)))
test_features[:, 1] = np.array(list(map(lambda x: order[x], address)))
test_features[:, 2] = np.array(list(map(lambda x: cumCustomer[x], address)))

In [27]:
normalize = layers.Normalization()
normalize.adapt(train_features)

In [28]:
model = tf.keras.Sequential([
  normalize,
  layers.Dense(64),
  layers.ReLU(),
  layers.Dense(512),
  layers.ReLU(),
  layers.Dense(512),
  layers.ReLU(),
  layers.Dense(32),
  layers.ReLU(),
  layers.Dense(1)
])

model.compile(loss = tf.keras.losses.MeanSquaredError(),
              optimizer = tf.keras.optimizers.Adam())

model.fit(train_features, train_labels, epochs=1000)

Epoch 1/1000
3/3 [==============================] - 1s 4ms/step - loss: 40250.8828
Epoch 2/1000
3/3 [==============================] - 0s 3ms/step - loss: 39901.9570
Epoch 3/1000
3/3 [==============================] - 0s 4ms/step - loss: 39338.0391
Epoch 4/1000
3/3 [==============================] - 0s 4ms/step - loss: 38309.6914
Epoch 5/1000
3/3 [==============================] - 0s 4ms/step - loss: 36693.1562
Epoch 6/1000
3/3 [==============================] - 0s 4ms/step - loss: 34209.2695
Epoch 7/1000
3/3 [==============================] - 0s 4ms/step - loss: 30521.8438
Epoch 8/1000
3/3 [==============================] - 0s 4ms/step - loss: 25719.2383
Epoch 9/1000
3/3 [==============================] - 0s 4ms/step - loss: 20578.0000
Epoch 10/1000
3/3 [==============================] - 0s 4ms/step - loss: 16240.1855
Epoch 11/1000
3/3 [==============================] - 0s 4ms/step - loss: 14202.5996
Epoch 12/1000
3/3 [==============================] - 0s 4ms/step - loss: 14531.2939
E

In [29]:
model.trainable = False

In [30]:
model.evaluate(test_features, test_labels)

1/1 [==============================] - 0s 96ms/step - loss: 1117.1073


1117.1072998046875

In [31]:
df_test = pd.DataFrame({'estimatedTime': test_labels, 'modelTime': np.array(model(test_features))[:, 0]})
df_test["pctError"] = abs((df_test.modelTime / df_test.estimatedTime) * 100 - 100)

df_test.head()

,estimatedTime,modelTime,pctError
42,45.0,49.953876,11.008614
19,359.0,267.660645,25.442717
3,126.0,106.866264,15.185504
89,64.0,49.953876,21.947068
96,103.0,106.866264,3.753655


In [32]:
df_test.replace([np.inf, -np.inf], 0, inplace=True)

In [33]:
df_test.pctError.mean()

13.369838879105714

In [34]:
model.save('./model/model.h5')